---
title: EDA - Wavelength Distribution
project: "total_dataset_EDA"
cdt: 2024-09-06T13:16:52
description: "Observation of wavelength distribution and determination of common cutoff"
conclusion: "175 samples start at 190nm. 104 include a max of 400nm, 71 run until 600 nm."
---


In [ ]:
import duckdb as db
import polars as pl

db_path = "/Users/jonathan/mres_thesis/wine_analysis_hplc_uv/wines.db"

con = db.connect(db_path)


What are the unique counts of sample per wavelength?

In [ ]:
# number of unique counts

samples_per_wavelength = con.sql(
    """
    SELECT
        wavelength,
        COUNT(DISTINCT ID) as num_samples
    FROM
        pbl.chromatogram_spectra_long
    GROUP BY
        wavelength
    ORDER BY
        num_samples DESC
    """
)

unique_sample_counts = con.sql(
    """
    SELECT
        DISTINCT num_samples as unique_sample_counts
    FROM
        samples_per_wavelength
    """
)

unique_sample_counts.pl()


Not sure how to interpret that one, 71 samples have wavelengths on top of the that the other 104 dont have?

Yes. Every samples wavelength starts at 190nm.

In [ ]:
wavelength_min_max_per_id = con.sql(
    """
    SELECT
        id,
        min(wavelength) as nm_min,
        max(wavelength) as nm_max,
    FROM
        pbl.chromatogram_spectra_long
    GROUP BY
        id
    """
)
wavelength_min_max_per_id.pl()


does every sample start at 190?

In [ ]:
unique_min_wavelengths = con.sql(
    """
    SELECT
        DISTINCT nm_min
    FROM
        wavelength_min_max_per_id
    """
)
unique_min_wavelengths.pl()


how many end wavelengths?

In [ ]:
unique_max_wavelengths = con.sql(
    """
    SELECT
        DISTINCT nm_max
    FROM
        wavelength_min_max_per_id
    """
)
unique_max_wavelengths.pl()


there are two wavelengths, 600 and 400.

In [ ]:
# how many per wavelength range?

samples_per_wavelength_max = con.sql(
    """
    SELECT
        nm_max,
        COUNT (id) count
    FROM
        wavelength_min_max_per_id
    GROUP BY
        nm_max
    ORDER BY
        nm_max
    """
)

samples_per_wavelength_max.pl()


104 


104 samples run from 190 to 400 nm. 71 samples run from 190 to 600 nm.

# Conclusion


All samples 175 samples in the total dataset (across 'raw' and 'cuprac') start at 190 nm and run to either 400 or 600 nm. 104 run until 400nm, and 71 include a range of 190 - 600 nm.